%md
## PONTIFICIA UNIVERSIDAD JAVERIANA ##

Materia: Procesamieno de datos a gran escala

Limpieza de datos Nivel educativo por municipio

In [1]:
import findspark
findspark.init()

import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib as plt
%matplotlib inline
from matplotlib import rcParams
import squarify
from sklearn.metrics import roc_curve, auc

#Biblioteca PySpark
import warnings
import pyspark
from pyspark.context import SparkContext
from pyspark.sql import SparkSession

from pyspark.sql import SQLContext
from pyspark.sql.types import *
from pyspark.sql.functions import input_file_name, mean, col, split, regexp_extract, when, lit, isnan, count, udf
from pyspark import SparkFiles
from pyspark.sql.types import *

#Bioblioteca ML para PySpark
from pyspark.ml import Pipeline
from pyspark.sql.functions import col, sum
#Importar el SO
from time import time
from pyspark.conf import SparkConf
import os

# Conexión a Spark

In [2]:
SPARK_MASTER_URL = os.getenv("SPARK_MASTER_URL", "spark://10.43.103.125:7077")
configura = SparkConf()
configura.setMaster(SPARK_MASTER_URL)
configura.set('spark.local.dir', '/almacen')
configura.setAppName("PrimerSparkGonzalez")
spark = SparkSession.builder.config(conf=configura).getOrCreate()
SQLContext(sparkContext=spark.sparkContext, sparkSession=spark)
# spark = spark.sparkContext

### Revisar si El servicio esta vivo


spark

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/11/10 22:08:30 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
24/11/10 22:08:30 WARN SparkConf: Note that spark.local.dir will be overridden by the value set by the cluster manager (via SPARK_LOCAL_DIRS in mesos/standalone/kubernetes and LOCAL_DIRS in YARN).


In [3]:
url = "https://raw.githubusercontent.com/Kahsma/Proyecto_PDGE/refs/heads/main/Datos/Nivel%20de%20educacion.csv"

spark.sparkContext.addFile(url)


In [4]:
NivelEducación = spark.read.csv("file://" + SparkFiles.get("Nivel de educacion.csv"), header = True , inferSchema = True )

In [5]:
NivelEducación.show(3)

24/11/10 22:08:43 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


+----+----------------+-------------+-------------------+------------+----------+-------------+--------------+-----------------------+--------------+-------------------------+-----------------------+-------------------------+--------------------+---------------+--------------------------+------------------------+--------------------------+---------------------+------------------------+---------------------------+---------+--------------------+------------------+--------------------+---------------+----------+---------------------+-------------------+---------------------+----------------+-----------+----------------------+--------------------+----------------------+-----------------+----------+---------------------+-------------------+---------------------+----------------+
| AÑO|CÓDIGO_MUNICIPIO|    MUNICIPIO|CÓDIGO_DEPARTAMENTO|DEPARTAMENTO|CÓDIGO_ETC|          ETC|POBLACIÓN_5_16|TASA_MATRICULACIÓN_5_16|COBERTURA_NETA|COBERTURA_NETA_TRANSICIÓN|COBERTURA_NETA_PRIMARIA|COBERTURA_NETA_S

In [6]:
# Calculate nulls for each column
null_counts = NivelEducación.select([sum(col(c).isNull().cast("int")).alias(c) for c in NivelEducación.columns])

# Show the result
null_counts.show()


+---+----------------+---------+-------------------+------------+----------+---+--------------+-----------------------+--------------+-------------------------+-----------------------+-------------------------+--------------------+---------------+--------------------------+------------------------+--------------------------+---------------------+------------------------+---------------------------+---------+--------------------+------------------+--------------------+---------------+----------+---------------------+-------------------+---------------------+----------------+-----------+----------------------+--------------------+----------------------+-----------------+----------+---------------------+-------------------+---------------------+----------------+
|AÑO|CÓDIGO_MUNICIPIO|MUNICIPIO|CÓDIGO_DEPARTAMENTO|DEPARTAMENTO|CÓDIGO_ETC|ETC|POBLACIÓN_5_16|TASA_MATRICULACIÓN_5_16|COBERTURA_NETA|COBERTURA_NETA_TRANSICIÓN|COBERTURA_NETA_PRIMARIA|COBERTURA_NETA_SECUNDARIA|COBERTURA_NETA_MEDIA

In [7]:
# Remove spaces in column names
for col_name in NivelEducación.columns:
    NivelEducación = NivelEducación.withColumnRenamed(col_name, col_name.strip())

# Show the DataFrame to confirm changes
NivelEducación.show(3)


+----+----------------+-------------+-------------------+------------+----------+-------------+--------------+-----------------------+--------------+-------------------------+-----------------------+-------------------------+--------------------+---------------+--------------------------+------------------------+--------------------------+---------------------+------------------------+---------------------------+---------+--------------------+------------------+--------------------+---------------+----------+---------------------+-------------------+---------------------+----------------+-----------+----------------------+--------------------+----------------------+-----------------+----------+---------------------+-------------------+---------------------+----------------+
| AÑO|CÓDIGO_MUNICIPIO|    MUNICIPIO|CÓDIGO_DEPARTAMENTO|DEPARTAMENTO|CÓDIGO_ETC|          ETC|POBLACIÓN_5_16|TASA_MATRICULACIÓN_5_16|COBERTURA_NETA|COBERTURA_NETA_TRANSICIÓN|COBERTURA_NETA_PRIMARIA|COBERTURA_NETA_S

In [8]:
# Select only the specified columns
columns_to_keep = ["CÓDIGO_MUNICIPIO", "TASA_MATRICULACIÓN_5_16", "COBERTURA_BRUTA", "SEDES_CONECTADAS_A_INTERNET"]
NivelEducación = NivelEducación.select(*columns_to_keep)

# Show the result to confirm
NivelEducación.show(10)


+----------------+-----------------------+---------------+---------------------------+
|CÓDIGO_MUNICIPIO|TASA_MATRICULACIÓN_5_16|COBERTURA_BRUTA|SEDES_CONECTADAS_A_INTERNET|
+----------------+-----------------------+---------------+---------------------------+
|           99773|                  49.85|          56.18|                       NULL|
|           99624|                  83.87|           93.1|                       NULL|
|           99524|                  95.94|         105.45|                       NULL|
|           99001|                 144.52|          158.5|                       NULL|
|           97889|                  47.95|          52.05|                       NULL|
|           97777|                    0.0|              0|                       NULL|
|           97666|                   22.6|          23.34|                       NULL|
|           97511|                  54.29|          71.39|                       NULL|
|           97161|                   51.3| 

In [9]:
# Drop rows with any null values
NivelEducación = NivelEducación.dropna()

# Show the result to confirm
NivelEducación.show(10)


+----------------+-----------------------+---------------+---------------------------+
|CÓDIGO_MUNICIPIO|TASA_MATRICULACIÓN_5_16|COBERTURA_BRUTA|SEDES_CONECTADAS_A_INTERNET|
+----------------+-----------------------+---------------+---------------------------+
|           85015|                   58.4|          65.23|                      11.11|
|           85125|                   88.9|         106.08|                      16.67|
|           85136|                   70.9|          76.63|                       12.5|
|           85139|                  122.5|         139.05|                      29.03|
|           85162|                   91.9|         108.29|                      84.62|
|           85225|                   78.2|          89.06|                       9.52|
|           85230|                  114.5|          132.5|                       20.0|
|           85250|                  121.9|         145.07|                       9.33|
|           85263|                  124.8| 

In [10]:
# Calculate nulls for each column
null_counts = NivelEducación.select([sum(col(c).isNull().cast("int")).alias(c) for c in NivelEducación.columns])

# Show the result
null_counts.show()


+----------------+-----------------------+---------------+---------------------------+
|CÓDIGO_MUNICIPIO|TASA_MATRICULACIÓN_5_16|COBERTURA_BRUTA|SEDES_CONECTADAS_A_INTERNET|
+----------------+-----------------------+---------------+---------------------------+
|               0|                      0|              0|                          0|
+----------------+-----------------------+---------------+---------------------------+



In [11]:
# Get the total count of records
total_records = NivelEducación.count()

# Display the result
print(f"Total number of records: {total_records}")


Total number of records: 7667


In [12]:
# Count the total number of records before removing duplicates
initial_count = NivelEducación.count()

# Remove duplicates based on 'CÓDIGO_MUNICIPIO' and keep the first occurrence
NivelEducación = NivelEducación.dropDuplicates(subset=["CÓDIGO_MUNICIPIO"])

# Count the total number of records after removing duplicates
final_count = NivelEducación.count()

# Calculate the number of duplicates removed
duplicates_removed = initial_count - final_count

# Display the result
print(f"Number of duplicate records removed: {duplicates_removed}")
print(f"Total records after removing duplicates: {final_count}")


Number of duplicate records removed: 6549
Total records after removing duplicates: 1118


In [13]:
# Define the output path for the CSV file
output_path = "Proyecto/"  # Replace with your desired directory path

# Write the DataFrame to a CSV file
NivelEducación.write.csv(output_path, header=True, mode="overwrite")
